In [1]:
import numpy as np
import scipy as sp
import pandas as pd

from scipy import optimize

%matplotlib inline 
import matplotlib.pyplot as plt

In [7]:
mean_S=[114.64866107858613, 281.263665030094, 434.1710447166398, 569.9988028399946, 687.0546718639996, 780.4870062669543, 864.119024879455, 916.7825679232428, 975.9915880272431, 1022.9939908123263]
rms=[114.74968236197647, 282.01041012696396, 434.47275315291114, 570.8325673130487, 687.4374797608367, 781.6874291625296, 864.5162056661065, 917.0785397347968, 976.4170138865792, 1024.7237580132255]
sigma_S=[4.813950047724185, 20.509074913742488, 16.188797420809703, 30.84128185045552, 22.93831837303278, 43.304387297040904, 26.20268499031733, 23.297450379212293, 28.82022045794128, 59.515334146904]
S_ref=[524.36037518,717.49813895,850.49419746, 715.4684653,951.54946654,893.63784202,811.12501887,896.93268679,870.97201082,1003.64425617]

In [3]:
def rndm(a, b, g, size=1):
    """Power-law gen for pdf(x)\propto x^{g-1} for a<=x<=b"""
    r = np.random.random(size=size)
    ag, bg = a**g, b**g
    return (ag + (bg - ag)*r)**(1./g)

N= 10000



def S_ref(A,B,E):
    return (E/A)**(1/B)


def S_i(alpha, beta, delta, cos, f):
    cos_ref=np.cos(38*np.pi/180.)**2
    x = cos-cos_ref
    return f*(delta+beta*x+alpha*x**2)

def S_i_fit(cos, alpha, beta, f):
    cos_ref=np.cos(38*np.pi/180.)**2
    x = cos-cos_ref
    return f*(alpha*x**2+beta*x+1)


def get_random_vars(N):
    E0 = 10**15
    E1 = 10**18
    gamma = -2.5

    A = 10**12
    B = 1.2
    b=0.919
    a=-1.13
    c=1

    E = rndm(E0, E1, gamma, N) 
    
    S_i_ref=S_ref(A,B,E)
    cos_2 = np.random.rand(N)
    S=S_i(a,b,c,cos_2,S_i_ref)
    data=pd.DataFrame()
    data['E'] = E
    data['S_ref'] = S_i_ref
    data['cos2'] = cos_2
    data['S'] = S
    data['th'] = np.arccos(np.sqrt(data.cos2))
    data['lgE'] = np.log10(data.E)
    data['lgS'] = np.log10(data.S)
    data['lgS_ref'] = np.log10(data.S_ref)
    data = data.sort_values(['lgS'])
    data['I'] = 0
    bins = np.linspace(0, 1, 11, endpoint = True )
    ind = np.digitize(data['cos2'],bins)
    groups = data.groupby(ind)
                      
    for name, group in groups:
        values = group['lgS'].apply(lambda x: group[group['lgS']>x].count())
        data.loc[group.I.index.tolist(), 'I']= values.I 
        
    data['sqrt_I']=np.sqrt(data.I)
    return data
b=0.919
a=-1.13
c=1

In [4]:
def lnlike(a,b,c, S_ref,lnf, x, y, yerr):
    model = S_ref*(a * x**2 + b*x+c)
    inv_sigma2 = 1.0/(yerr**2 + model**2*np.exp(2*lnf))
    return -0.5*(np.sum((y-model)**2*inv_sigma2 - np.log(inv_sigma2)))

In [5]:
#from simul_functions import S_i

In [24]:
x=np.linspace(0.05,0.95,10, endpoint = True)
y=S_i_fit(x,a,b,S_ref)
print(type(c))

<class 'int'>


In [28]:
import scipy.optimize as op
nll = lambda *args: -lnlike(*args)
x0=[a, b, c, np.log(np.asarray(mean_S)) ]
print(type(y))
result = op.minimize(nll, np.asarray(x0), args=(x, y, np.asarray(sigma_S)))
m_ml, b_ml, lnf_ml = result["x"]

<class 'numpy.ndarray'>


ValueError: setting an array element with a sequence.